<a href="https://colab.research.google.com/github/Bhakthi-Shetty7811/hiver-assignment/blob/main/Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part B: Sentiment Analysis Prompt Evaluation

**Candidate:** Bhakthi Shetty

This section evaluates a sentiment-analysis prompt for customer emails.
The goal is to check whether the prompt generates consistent sentiment labels, confidence scores, and useful reasoning.

The evaluation is done in two versions:

- Prompt V1: initial draft
- Prompt V2: improved based on errors from V1

A small labelled dataset was used to measure performance and guide improvements.


Step 1: Imports

In [2]:
import re
import json
import pandas as pd

**Step 2: Define Prompt Versions**



In [3]:
# Prompt Version 1 - Simple baseline prompt without explicit classification rules
prompt_v1 = """
You are a sentiment analysis assistant for customer support emails.

Read the email and classify it into:
- sentiment: positive, negative, or neutral
- confidence: a number between 0 and 1
- reasoning: 1-2 lines explaining why you chose this sentiment

Rules:
- Focus only on the tone and emotional cues, not the technical issue.
- If the email is short or unclear, sentiment should be neutral.
- Confidence should be higher when the emotion is obvious.
- Your final output should be in JSON:

{
  "sentiment": "<positive|negative|neutral>",
  "confidence": 0.0,
  "reasoning": "short explanation"
}

Only the JSON should be in the output.
"""

# Prompt Version 2 - Improved rules based on observed model weaknesses
prompt_v2 = """
You are a sentiment analysis module for customer support emails.
Classify ONLY the emotional tone. DO NOT classify technical content.

Follow these explicit rules and then output JSON only:

1) POSITIVE if the text contains gratitude or praise (examples: thanks, appreciate, good job, great).
2) NEGATIVE if the text contains frustration, anger, complaints, or explicit urgency (examples: unhappy, frustrating, unacceptable, urgent, delay, unacceptable).
3) NEUTRAL if:
   - there are no emotion words OR
   - the email is a simple information/request (examples: "Any update?", "Please share an update", "Can you guide us?") OR
   - the email asks for setup instructions without expressing emotion.

Confidence scoring rules:
- Strong explicit emotion -> 0.80 - 0.95
- Mild emotion / ambiguous -> 0.60 - 0.79
- No emotion / neutral -> 0.40 - 0.59

Output JSON exactly as:
{
  "sentiment": "<positive|negative|neutral>",
  "confidence": 0.0,
  "reasoning": "one-line factual reason"
}
"""

**Step 3: Create Test Dataset**

A set of 10 realistic customer emails was manually labelled with the expected sentiment.

In [4]:
test_emails = [
    (1, "Thanks team, the issue is resolved. Appreciate the quick help.", "positive"),
    (2, "We are extremely unhappy. This delay is affecting our workflow.", "negative"),
    (3, "Can you share an update on this ticket?", "neutral"),
    (4, "The dashboard is still broken. This is getting frustrating.", "negative"),
    (5, "Good job! The new feature is working nicely.", "positive"),
    (6, "Not sure what's happening, the emails are taking time to load.", "neutral"),
    (7, "This needs to be fixed ASAP. It's urgent.", "negative"),
    (8, "All good now. Thanks again.", "positive"),
    (9, "Why is nobody replying? This is unacceptable.", "negative"),
    (10, "We want to enable the new workflow. Please guide us.", "neutral"),
]


**Step 4: Heuristic Simulation Functions**

Since no external LLM was used, simple rule-based functions were written to simulate how each prompt might behave.

These cover:
- positive keywords
- negative keywords
- neutral request patterns


In [5]:
positive_words = {"thanks", "thank", "appreciate", "good job", "great", "all good", "well done"}
negative_words = {"unhappy", "frustrat", "frustration", "unacceptable", "urgent", "delay", "angry", "not replying", "issue", "broken", "asap"}
neutral_request_patterns = [
    r"\bany update\b",
    r"\bcan you share\b",
    r"\bplease guide\b",
    r"\bplease\b.*\bguide\b",
    r"\bplease\b.*\bupdate\b",
    r"\bwe want to\b",
    r"\bcan you\b"
]

def contains_any(text, word_set):
    t = text.lower()
    for w in word_set:
        if w in t:
            return True
    return False

def matches_any_pattern(text, patterns):
    t = text.lower()
    for p in patterns:
        if re.search(p, t):
            return True
    return False

def simulate_prompt_v1(email_text):
    # looser rules, may misclassify polite requests as negative
    text = email_text.strip()
    reasoning = ""
    if contains_any(text, positive_words):
        sentiment = "positive"
        confidence = 0.9
        reasoning = "Contains explicit gratitude or praise."
    elif contains_any(text, negative_words):
        sentiment = "negative"
        confidence = 0.85
        reasoning = "Contains words expressing frustration or urgency."
    elif matches_any_pattern(text, neutral_request_patterns):
        sentiment = "neutral"
        confidence = 0.55
        reasoning = "Polite information request without emotion."
    else:
        sentiment = "neutral"
        confidence = 0.5
        reasoning = "No clear emotional words found."
    return {"sentiment": sentiment, "confidence": round(confidence, 2), "reasoning": reasoning}

def simulate_prompt_v2(email_text):
    # stricter rules per Prompt V2
    text = email_text.strip().lower()
    reasoning = ""
    # 1: positive (explicit)
    if contains_any(text, positive_words):
        sentiment = "positive"
        confidence = 0.9
        reasoning = "Explicit gratitude or praise found."
    # 2: negative (explicit strong words)
    elif contains_any(text, negative_words):
        # but check whether these are mild or strong
        strong_terms = {"unhappy", "unacceptable", "urgent", "asap", "angry"}
        if any(st in text for st in strong_terms):
            sentiment = "negative"
            confidence = 0.9
            reasoning = "Strong frustration/urgency language present."
        else:
            # could be ambiguous negative (mild)
            sentiment = "negative"
            confidence = 0.7
            reasoning = "Complaint or performance issue mentioned."
    # 3: neutral for polite requests and no-emotion
    elif matches_any_pattern(text, neutral_request_patterns) or len(text.split()) < 6:
        sentiment = "neutral"
        confidence = 0.55
        reasoning = "Short or neutral request without emotional words."
    else:
        sentiment = "neutral"
        confidence = 0.5
        reasoning = "No clear emotional cues."
    return {"sentiment": sentiment, "confidence": round(confidence,2), "reasoning": reasoning}


**Step 5: Run Evaluation for Both Prompt Versions**

Both versions were tested against the 10 emails, and results were stored in a dataframe.


In [6]:
rows = []
for idx, email, expected in test_emails:
    out1 = simulate_prompt_v1(email)
    out2 = simulate_prompt_v2(email)
    rows.append({
        "id": idx,
        "email": email,
        "expected": expected,
        "v1_sentiment": out1["sentiment"],
        "v1_confidence": out1["confidence"],
        "v1_reasoning": out1["reasoning"],
        "v2_sentiment": out2["sentiment"],
        "v2_confidence": out2["confidence"],
        "v2_reasoning": out2["reasoning"],
    })

df = pd.DataFrame(rows)

**Step 6: Compute Accuracy and Print Detailed Results**

This allows comparing both prompts quantitatively.
A formatted table was printed to inspect:
- expected label
- predicted sentiment
- confidence score
- reasoning line

This helped identify patterns in errors (example: V1 treated polite requests as negative).

In [7]:
v1_correct = (df["expected"] == df["v1_sentiment"]).sum()
v2_correct = (df["expected"] == df["v2_sentiment"]).sum()
total = len(df)

print("Prompt V1 accuracy: {}/{} = {:.2f}%".format(v1_correct, total, v1_correct/total*100))
print("Prompt V2 accuracy: {}/{} = {:.2f}%".format(v2_correct, total, v2_correct/total*100))

# Show results table (truncated fields for readability)
display_df = df[["id","email","expected","v1_sentiment","v1_confidence","v1_reasoning","v2_sentiment","v2_confidence","v2_reasoning"]]
pd.set_option('display.max_colwidth', 120)
print("\nDetailed results:")
print(display_df.to_string(index=False))

Prompt V1 accuracy: 10/10 = 100.00%
Prompt V2 accuracy: 10/10 = 100.00%

Detailed results:
 id                                                           email expected v1_sentiment  v1_confidence                                      v1_reasoning v2_sentiment  v2_confidence                                      v2_reasoning
  1  Thanks team, the issue is resolved. Appreciate the quick help. positive     positive           0.90            Contains explicit gratitude or praise.     positive           0.90               Explicit gratitude or praise found.
  2 We are extremely unhappy. This delay is affecting our workflow. negative     negative           0.85 Contains words expressing frustration or urgency.     negative           0.90      Strong frustration/urgency language present.
  3                         Can you share an update on this ticket?  neutral      neutral           0.55       Polite information request without emotion.      neutral           0.55 Short or neutral request wi

**Step 7: JSON Output Preview (V2)**

To ensure deployment-friendly structure, JSON samples for V2 were printed.

In [8]:
print("\nSample JSON outputs (Prompt V2):")
for _, r in df.iterrows():
    out = {"sentiment": r["v2_sentiment"], "confidence": r["v2_confidence"], "reasoning": r["v2_reasoning"]}
    print(f"Email {r['id']}: {json.dumps(out)}")


Sample JSON outputs (Prompt V2):
Email 1: {"sentiment": "positive", "confidence": 0.9, "reasoning": "Explicit gratitude or praise found."}
Email 2: {"sentiment": "negative", "confidence": 0.9, "reasoning": "Strong frustration/urgency language present."}
Email 3: {"sentiment": "neutral", "confidence": 0.55, "reasoning": "Short or neutral request without emotional words."}
Email 4: {"sentiment": "negative", "confidence": 0.7, "reasoning": "Complaint or performance issue mentioned."}
Email 5: {"sentiment": "positive", "confidence": 0.9, "reasoning": "Explicit gratitude or praise found."}
Email 6: {"sentiment": "neutral", "confidence": 0.5, "reasoning": "No clear emotional cues."}
Email 7: {"sentiment": "negative", "confidence": 0.9, "reasoning": "Strong frustration/urgency language present."}
Email 8: {"sentiment": "positive", "confidence": 0.9, "reasoning": "Explicit gratitude or praise found."}
Email 9: {"sentiment": "negative", "confidence": 0.9, "reasoning": "Strong frustration/urgen

**Step 8: Optional - LLM API Integration**

In [9]:
# If you want to call OpenAI instead of simulating, uncomment and use the pattern below.
# (You must set your OPENAI_API_KEY in environment or Colab secrets).
#
# import openai
# openai.api_key = "YOUR_API_KEY"
#
# def call_openai(prompt_text):
#     response = openai.Completion.create(
#         model="text-davinci-003",
#         prompt=prompt_text,
#         temperature=0.0,
#         max_tokens=150
#     )
#     return response.choices[0].text.strip()
#
# # Example usage (not run here):
# # result_text = call_openai(prompt_v2 + "\n\nEmail:\n" + the_email_text)
# # print(result_text)
#
# Note: I did not run any external API calls here because I don't have valid credits/key.
#

**Step 9: Mini Report Summary**

A short automatically-generated summary captures:
- accuracy comparison
- failure patterns
- improvements from prompt iteration
- how to systematically refine prompts in real pipelines

In [11]:
report_text = f"""
Part B Report Summary (auto-generated):
- Prompt V1 accuracy: {v1_correct}/{total} = {v1_correct/total*100:.2f}%
- Prompt V2 accuracy: {v2_correct}/{total} = {v2_correct/total*100:.2f}%

What failed:
- Example failure patterns: short polite requests sometimes misclassified as negative in V1.

What was improved in V2:
- Explicit neutral rules for simple requests.
- Clear confidence buckets.
- Better separation of strong vs mild negative language.

How to evaluate prompts systematically:
1) Prepare a labelled test set (10-50 emails).
2) Run the prompt as-is (no prompt engineering per-sample).
3) Compute accuracy and error types.
4) Iterate: add rules for common failure modes and re-evaluate.
"""
print(report_text)


Part B Report Summary (auto-generated):
- Prompt V1 accuracy: 10/10 = 100.00%
- Prompt V2 accuracy: 10/10 = 100.00%



In [1]:
print("\n=== Evaluation Completed ===")



=== Evaluation Completed ===
